## Reviews Processing App

### Import Libraries

In [128]:
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, log_loss, classification_report
from sklearn.model_selection import train_test_split
from IPython.display import display, clear_output
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
import contractions
print("Import successful!")

Import successful!


### Import Data

In [202]:
# Columns X = [title, content] y = label (0 = negative, 1 = positive)
train_data = pd.read_csv('data/train.csv', nrows=100000)
test_data = pd.read_csv('data/test.csv')

# Print the counts of 0 and 1 labels in the training data
label_counts = train_data['label'].value_counts()
print(label_counts)

# Selected columns x = review y = label
train_data = train_data[['label', 'title']].rename(columns={'title': 'review'})
test_data = test_data[['label', 'title']].rename(columns={'title': 'review'})


label
1    51267
0    48733
Name: count, dtype: int64


### Data Preprocessing

In [ ]:
# Load the spaCy model globally for reuse
nlp = spacy.load("en_core_web_sm")

# Define stop words and punctuation once
keep_words = {"not", "no", "never", "very", "always", "too", "hardly", "barely"}
custom_stop_words = set(nlp.Defaults.stop_words - keep_words)
punctuation_set = set(string.punctuation)

def preprocess_text(text):
    # Early return for None or empty strings
    if text is None:
        return ""

    # Ensure text is a string
    text = str(text)

    # Convert to lowercase
    text = text.lower()

    # Expand abbreviations
    text = contractions.fix(text)

    # Process text with spaCy
    doc = nlp(text)

    # Collect preprocessed tokens
    preprocessed_tokens = [
        re.sub(r'\W+', '', token.lemma_)  # Clean token
        for token in doc
        if token.text not in custom_stop_words  # Exclude custom stop words
        and token.text not in punctuation_set  # Exclude punctuation
        and token.lemma_.strip()  # Exclude empty lemmas
    ]

    return " ".join(filter(None, preprocessed_tokens))

# Apply the function on the dataset
train_data['review'] = train_data['review'].apply(preprocess_text)

### Training

In [178]:
reviews = train_data['review']
labels = train_data['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Feature extraction using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a classification model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

### Evaluation

In [196]:
# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred))

 18/625 ━━━━━━━━━━━━━━━━━━━━ 2:18 228ms/step

KeyboardInterrupt: 

### Manual Testing

In [188]:
prediction = model.predict(vectorizer.transform([preprocess_text("This product don't suck")]))
print('positive' if prediction[0] == 1 else 'negative')

negative
